## Example query using some advanced features

In [ ]:
import os
import sys
import json
import requests
from requests.auth import HTTPBasicAuth

# Optional (if coordinates not in decimal degrees)
from astropy import units as u
from astropy.coordinates import SkyCoord

### 1. Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example, in an open shell terminal type: <code>export SLED_USERNAME=<your_SLED_username></code> and <code>export SLED_PASSWORD=<your_SLED_password></code>. You can also include these two commands in your shell profile file (e.g. ~/.bash_profile in Unix/Linux/Mac) or in an environment file.

In [ ]:
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')
if (not username) or (not password):
    print('You have not set your SLED username and password environment variables.')

### 2. Define the query parameters

Coordinates should be in decimal degrees and in the J2000 system. To convert from ra/dec in hours-minutes/degrees-minutes, you can use the following astropy coords functionality. This example query consists of:
- a box search on the COSMOS field,
- confirmed lenses or contaminants,
- and lenses with observations with the ACS instrument

The data we choose to download are:
- Lens properties: the RA, DEC, name, and mughsot images
- Related data: redshfits

In [ ]:
#COSMOS field
ra_min, ra_max = 149.4114, 150.8269
dec_min, dec_max = 1.4988, 2.9128

## If the RA and DEC are not in decimal degrees then use the following loop
#ra_min  = SkyCoord(ra_min, unit=(u.hourangle, u.deg))
#ra_max  = SkyCoord(ra_max, unit=(u.hourangle, u.deg))
#dec_min = SkyCoord(dec_min, unit=(u.hourangle, u.deg))
#dec_max = SkyCoord(dec_max, unit=(u.hourangle, u.deg))


query_pars = {
    'lens-ra_min':round(ra_min, 4),
    'lens-ra_max':round(ra_max, 4),
    'lens-dec_min':round(dec_min, 4),
    'lens-dec_max':round(dec_max, 4),
    'lens-flag': [
        'CONFIRMED',
        'CONTAMINANT'
    ],
    'imaging-instrument': [
        1,
    ],
    "download-lens_options": [
        "ra",
        "dec",
        "name",
        "mugshot"
    ],
    "download-related": [
        "redshift",
    ],
}

### 3. Now we query the database

Use the requests package to send a POST request to the relevant API endpoint. The response is parsed as json and converted to a dictionary. There is no need to change the following lines.


In [ ]:
print("Querying the SLED server...")
r = requests.post("https://sled.amnh.org/api/query-lenses-full/",
                  data=json.dumps(query_pars),
                  headers={'Content-Type':'application/json'},
                  auth=HTTPBasicAuth(username,password)
                  )
dbquery = json.loads(r.text)

### 4. Understanding the response

The response is a dictionary that contains two keys:

- *'errors'*: any errors from the given query parameter fields
- *'lenses'*: a list of all the corresponding lenses and the selected fields to download from SLED.

In [ ]:
if len(dbquery["errors"]) > 0:
    print(dbquery["errors"])
else:        
    print("Returned lenses: ",len(dbquery["lenses"]))
    print(dbquery["lenses"][0])